In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#cd into your home directory
cd ~/

/home/qiime2/Documents/Subthirty


In [9]:
print("Start")
#make sure your raw sequence files are in a directory named Raw
#see input-path
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path Raw \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza
!qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv

Start
Saved Visualization to: demux-paired-end.qzv


In [4]:
print("Dada2")
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza\
  --p-trunc-len-r 195\
  --p-trunc-len-f 215\
  --p-chimera-method none\
  --p-n-threads 0
!mv rep-seqs-dada2.qza rep-seqs.qza
!mv table-dada2.qza table.qza

Dada2
Saved FeatureTable[Frequency] to: table-dada2.qza
Saved FeatureData[Sequence] to: rep-seqs-dada2.qza
Saved SampleData[DADA2Stats] to: stats-dada2.qza


In [5]:
print("Feature tables")
#Make sure the metadata file is in the directory
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file HPMMMeta.txt
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

Feature tables
Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv


In [6]:
print("Uchime")
!qiime vsearch uchime-denovo \
  --i-table table.qza \
  --i-sequences rep-seqs.qza \
  --output-dir uchime-dn-out

Uchime
Saved FeatureData[Sequence] to: uchime-dn-out/chimeras.qza
Saved FeatureData[Sequence] to: uchime-dn-out/nonchimeras.qza
Saved UchimeStats to: uchime-dn-out/stats.qza


In [7]:
print("Uchime visual")
!qiime metadata tabulate \
  --m-input-file uchime-dn-out/stats.qza \
  --o-visualization uchime-dn-out/stats.qzv

Uchime visual
Saved Visualization to: uchime-dn-out/stats.qzv


In [8]:
!qiime feature-table filter-features \
  --i-table table.qza \
  --m-metadata-file uchime-dn-out/nonchimeras.qza \
  --o-filtered-table uchime-dn-out/table-nonchimeric-wo-borderline.qza
!qiime feature-table filter-seqs \
  --i-data rep-seqs.qza \
  --m-metadata-file uchime-dn-out/nonchimeras.qza \
  --o-filtered-data uchime-dn-out/rep-seqs-nonchimeric-wo-borderline.qza
!qiime feature-table summarize \
  --i-table uchime-dn-out/table-nonchimeric-wo-borderline.qza \
  --o-visualization uchime-dn-out/table-nonchimeric-wo-borderline.qzv

Saved FeatureTable[Frequency] to: uchime-dn-out/table-nonchimeric-wo-borderline.qza
Saved FeatureData[Sequence] to: uchime-dn-out/rep-seqs-nonchimeric-wo-borderline.qza
Saved Visualization to: uchime-dn-out/table-nonchimeric-wo-borderline.qzv


In [9]:
!mv uchime-dn-out/rep-seqs-nonchimeric-wo-borderline.qza rep-seqs.qza
!mv uchime-dn-out/table-nonchimeric-wo-borderline.qza table.qza

In [10]:
print("Aligning")
!qiime alignment mafft \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza
!qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza

Aligning
Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza


In [11]:
print("Building Tree")
!qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza
!qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Building Tree
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


In [12]:
print("Alpha rarefaction")
#look at demux plot overview for pmax
#use alpha rarefaction for determing rarefaction level
!qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 40000 \
  --m-metadata-file HPMMMeta.txt \
  --o-visualization alpha-rarefaction.qzv

Alpha rarefaction
Saved Visualization to: alpha-rarefaction.qzv


In [14]:
print("Classifying")
!qiime feature-classifier classify-sklearn \
  --i-classifier silva-132-99-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

Classifying
Plugin error from feature-classifier:



Debug info has been saved to /tmp/qiime2-q2cli-err-p5yjm9y4.log


In [15]:
!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

There was an issue with loading the file taxonomy.qza as metadata:

  Metadata file path doesn't exist, or the path points to something other than a file. Please check that the path exists, has read permissions, and points to a regular file (not a directory): taxonomy.qza

  There may be more errors present in the metadata file. To get a full report, sample/feature metadata files can be validated with Keemei: https://keemei.qiime2.org

  Find details on QIIME 2 metadata requirements here: https://docs.qiime2.org/2018.8/tutorials/metadata/



In [16]:
print("Filter out mitochondria and chloroplast")
!qiime taxa filter-table \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-exclude mitochondria,chloroplast \
  --o-filtered-table table.qza

Filter out mitochondria and chloroplast
Usage: qiime taxa filter-table [OPTIONS]

Error: Invalid value for "--i-taxonomy": Path "taxonomy.qza" does not exist.


In [22]:
!qiime tools export \
  --input-path table.qza \
  --output-path exported 

Error: no such option: --output-dir


In [23]:
!qiime tools export \
   --input-path taxonomy.qza \
   --output-path exported

Error: no such option: --output-dir


In [ ]:
#go into taxonomy.tsv and change headers: #OTUID taxonomy confidience 

In [5]:
!biom add-metadata -i exported/feature-table.biom -o table-with-taxonomy.biom --observation-metadata-fp exported/taxonomy.tsv --sc-separated taxonomy